# Facebook API
In this notebook I'll be interacting with Facebook's Graph API Explorer to find insight on Facebook pages. As a premise, a friend is about to launch an ecommerce website and this notebook will help me become familiarized with the API to help with marketing research on what successful facebook pages look like.

For this exercise I'll be pulling data from Steam's Facebook page.

In [1]:
import requests
import time
import pickle
import random
import string

from operator import itemgetter
import pandas as pd
import numpy as np

In [2]:
# My user token
token = 'EAACEdEose0cBAB8tZA0bH6OeZCrOT9IyFlS02wZCP6HZAWq1g6qJsImd22JVjulcdPLjUtMAMyh5OabmZAXnK7ttKbtYCU8IyxAOWZBWEdQZAKm6XQbgmFZBJ2OyBIhtJiiXlMSfmXVi9McJ0IFh7mRYwVafmZBYJSirJtfTeod4SLXNmkngftOTBEeeTFGq9NAKNl9s4wcDaGQZDZD'

In [3]:
def req_facebook(req):
    '''This function takes the request string and returns the response'''
    r = requests.get('https://graph.facebook.com/v2.8/' + \
                    req, {'access_token': token})
    return r

In [34]:
# Read json for steam post likes
result = req_facebook('Steam/likes?fields=posts{comments.limit(0).summary(true),likes.limit(0).summary(true)}').json()

# Save result as only the posts
# result = result['posts']

In [37]:
# Store the post likes in the data list
data = []

i = 0

while True:
    try:
        # sleeper so we don't attack fb!
        time.sleep(random.randint(2, 5))
        # extend appends a list into another list
        data.extend(result['data'])
        result = requests.get(result['paging']['next']).json()
        i += 1
        
        if i > 5:
            break
        
    except:
        print('done')
        break
        
# Save data using pickle
pickle.dump(data, open('steam_data.pkl', 'wb'))

done


In [43]:
data

[{'id': '159017290800264',
  'posts': {'data': [{'comments': {'data': [],
      'summary': {'can_comment': True, 'order': 'ranked', 'total_count': 6}},
     'id': '159017290800264_1559872454048067',
     'likes': {'data': [],
      'summary': {'can_like': True, 'has_liked': False, 'total_count': 21}}},
    {'comments': {'data': [],
      'summary': {'can_comment': True, 'order': 'ranked', 'total_count': 14}},
     'id': '159017290800264_1508352272533419',
     'likes': {'data': [],
      'summary': {'can_like': True, 'has_liked': False, 'total_count': 18}}},
    {'comments': {'data': [],
      'summary': {'can_comment': True, 'order': 'ranked', 'total_count': 69}},
     'id': '159017290800264_1478562232179090',
     'likes': {'data': [],
      'summary': {'can_like': True, 'has_liked': False, 'total_count': 534}}},
    {'comments': {'data': [],
      'summary': {'can_comment': True, 'order': 'ranked', 'total_count': 6}},
     'id': '159017290800264_1455622724473041',
     'likes': {'da

In [41]:
loaded_data = pickle.load(file=open('steam_data.pkl', 'rb'))

In [42]:
df = pd.io.json.json_normalize(loaded_data)
df.head()

,id,posts.data,posts.paging.next,posts.paging.previous
0,159017290800264,"[{'id': '159017290800264_1559872454048067', 'l...",https://graph.facebook.com/v2.8/15901729080026...,https://graph.facebook.com/v2.8/15901729080026...
1,106876872711112,"[{'id': '106876872711112_1734843813247735', 'l...",https://graph.facebook.com/v2.8/10687687271111...,https://graph.facebook.com/v2.8/10687687271111...
2,84296203938,"[{'id': '84296203938_10153880204953939', 'like...",https://graph.facebook.com/v2.8/84296203938/po...,https://graph.facebook.com/v2.8/84296203938/po...
3,72792772943,"[{'id': '72792772943_10152464714687944', 'like...",https://graph.facebook.com/v2.8/72792772943/po...,https://graph.facebook.com/v2.8/72792772943/po...
4,72318222746,"[{'id': '72318222746_10150600367332747', 'like...",https://graph.facebook.com/v2.8/72318222746/po...,https://graph.facebook.com/v2.8/72318222746/po...


In [8]:
df.drop(['comments.data', 'comments.summary.can_comment', \
        'comments.summary.order'], axis=1, inplace=True)
df.head()

,comments.summary.total_count,id
0,42,67919847338_10155311198912339
1,50,67919847338_10155308838982339
2,63,67919847338_10155306544662339
3,118,67919847338_10155304469922339
4,56,67919847338_10155304314382339


In [9]:
df.rename(columns={'comments.summary.total_count': 'total_comments'},\
             inplace=True)
df.head()

,total_comments,id
0,42,67919847338_10155311198912339
1,50,67919847338_10155308838982339
2,63,67919847338_10155306544662339
3,118,67919847338_10155304469922339
4,56,67919847338_10155304314382339
